# Sprint 16: Web scraping

#### Imports

In [2]:
import requests
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

## Level 1

### Exercise 1

#### Web scraping of the Madrid stock exchange with BeautifulSoup and Selenium

### BeautifulSoup

Tool for pulling out information from a webpage. Extracting tables, lists, paragraphs from single url. Perfect for small projects.

In [13]:
#scrape html content
url = "https://www.bolsamadrid.es"
page = requests.get(url)

##### Playing around with BeautifulSoup

In [14]:
#parse html code with beautiful soup
soup = BeautifulSoup(page.content, "html.parser")

In [15]:
#find elements by ID
results = soup.find(id="PortBolsa")

#results fomat pretty
#results.prettify()

In [16]:
#find elements by class
column = results.find_all("div", class_="coln1")
#for coln1 in column:
    #print(coln1, end="\n"*2)

In [17]:
#pick out child elements
for coln1 in column:
    element1 = coln1.findAll("div", class_="mclick")
    #print(element1)

In [20]:
#find child div element
for x in element1:
    element2 = x.find("div")
#print(element2)    

In [19]:
#print out just text
print(element2.text)

Los índices bursátiles como referencia en tiempos de crisis


In [22]:
#find elements by class name and text content
#find all elements with text crisis
crisis = results.find_all("div", string=lambda text: "crisis")
#how many maching jobs we have?
print(len(crisis))

85


In [23]:
#fetching url elements
urls = []

for w in crisis:
    links = w.find_all("a")
    for link in links:
        urls.append(link.get('href'))
        link_url = link["href"] 
        #print(f"URL: {link_url}\n") 

In [24]:
urls[:2]

['https://www.bolsasymercados.es/esp/Sala-Comunicacion/Nota-Prensa/20211122/nota_20211122_1/Javier_Hernani_y_Thomas_Zeeb_destacan_el_papel_de_la_Bolsa_como_punto_de_encuentro_con_America_Latina',
 'https://youtu.be/lzIVRMtoJQg']

##### Search for Indices Summary

In [20]:
link = soup.find('a', string='Resumen de Índices')
new_url = link.get('href')
new_url

'/esp/aspx/Indices/Resumen.aspx'

In [21]:
link = url+new_url
link

'https://www.bolsamadrid.es/esp/aspx/Indices/Resumen.aspx'

In [22]:
html_indices = requests.get(link)
soup_indices = BeautifulSoup(html_indices.content, 'html.parser')

In [23]:
table_indices = soup_indices.find("table",class_="TblPort")
table_indices.find_all("th")

[<th scope="col">Nombre</th>,
 <th scope="col">Anterior</th>,
 <th scope="col">Último</th>,
 <th scope="col">% Dif.</th>,
 <th scope="col">Máximo</th>,
 <th scope="col">Mínimo</th>,
 <th scope="col">Fecha</th>,
 <th scope="col">Hora</th>,
 <th class="Ult" scope="col">% Dif.<br/>Año 2021</th>]

In [24]:
columns=[]
for element in table_indices.find_all("th"):
    columns.append(element.get_text())
print(columns)

['Nombre', 'Anterior', 'Último', '% Dif.', 'Máximo', 'Mínimo', 'Fecha', 'Hora', '% Dif.Año 2021']


In [25]:
results=[]
for element in table_indices.find_all("td"):
    results.append(element.get_text())

In [26]:
data = {}
for i in range(9):
    data[columns[i]]=results[i::9]

df = pd.DataFrame(data)

In [27]:
df

,Nombre,Anterior,Último,% Dif.,Máximo,Mínimo,Fecha,Hora,% Dif.Año 2021
0,IBEX 35®,"8.402,70","8.455,20","0,62","8.562,00","8.431,70",29/11/2021,17:38:00,"4,73"
1,IBEX 35® con Dividendos,"25.846,10","26.007,70","0,63","26.336,20","25.935,40",29/11/2021,17:38:00,"7,20"
2,IBEX MEDIUM CAP®,"12.992,60","13.071,80","0,61","13.183,00","13.029,90",29/11/2021,17:38:00,"2,80"
3,IBEX SMALL CAP®,"7.862,50","7.874,30","0,15","7.966,40","7.866,60",29/11/2021,17:38:00,"-2,76"
4,IBEX 35® Bancos,"435,20","434,30","-0,21","445,90","433,40",29/11/2021,17:38:00,"15,47"
...,...,...,...,...,...,...,...,...,...
73,Índice ITX Inverso X3,"199,40","190,30","-4,56","196,60","180,90",29/11/2021,17:38:00,"-62,81"
74,Índice TEF Inverso X5,"12.036,30","11.197,90","-6,97","12.098,60","10.759,20",29/11/2021,17:38:00,"440,67"
75,Índice SAN Inverso X5,"5.838,10","5.818,30","-0,34","5.902,20","5.057,80",29/11/2021,17:38:00,"940,42"
76,Índice BBVA Inverso X5,"11.933,10","12.438,90","4,24","12.451,20","10.339,60",29/11/2021,17:38:00,"40,04"


In [28]:
df.to_csv('Indices.csv')

### Selenium

Tool for pulling out information from a webpage. Simulates a real browser and reads also JavaScript. Testing of web applications and midium sized projects.

In [29]:
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By 

opts = Options()
#operating in headless mode
opts.headless = True
browser = Firefox(options=opts)
browser.get('https://www.bolsamadrid.es')

In [31]:
#query the DOM - Document Object Model
#fetching elements in table IBEX35
ibex35 = browser.find_element(By.CLASS_NAME, "TblAccPort")
print(ibex35.text)

Nombre Último % Dif.  
ACCIONA 159,0000 -0,19
ACERINOX 10,1250 0,15
ACS 21,5400 2,47
AENA 129,3500 0,70
ALMIRALL 10,7900 0,37


In [32]:
#fetching elements in table Daily schedule
schedule = browser.find_element(By.CLASS_NAME, "TblPort")
print(schedule.text)

Evento/Indicador País Periodo Anterior Previsto Actual
Ventas al por menor  Japón Octubre -0,5% (r) 1,1% 0,9%
IPC armonizado provisional  España Noviembre 5,4% 5,6% 5,6%
IPC provisional  España Noviembre 5,4% 4,0% 5,6%
Precios industriales  Italia Octubre 13,3% 20,4%
Confianza del consumidor final  UEM Noviembre -4,8 (r) -6,8 -6,8
IPC provisional  Alemania Noviembre 4,5% 5,0% 5,2%
IPC armonizado provisional  Alemania Noviembre 4,6% 5,5% 6,0%
Ventas pendientes de viviendas  EEUU Octubre -2,4% (r) 0,9% 7,5%


In [33]:
#search for link to dividents
links = browser.find_element(By.LINK_TEXT, "Dividendos")
links.click()

In [34]:
#search for id of table
table = browser.find_element(By.ID, "ctl00_Contenido_tblDatos")

In [35]:
print(table.text)

Fecha
Descuento Fecha
Abono Emisora Valor ISIN Valor Ejercicio Tipo Importe Bruto
21/12/2021 23/12/2021 CLINICA BAVIERA, S.A. CLINICA BAVIERA ES0119037010 A cuenta 2021 Ord. (*)
20/12/2021 22/12/2021 VISCOFAN, S.A. VISCOFAN ES0184262212 A cuenta 2021 Ord. (*)
17/12/2021 21/12/2021 ENAGAS, S.A. ENAGAS ES0130960018 A cuenta 2021 Ord. (*)
17/12/2021 29/12/2021 BRADESPAR,S.A. BRADESPAR ORDINARIAS BRBRAPACNOR5 A cuenta 2021 Ord. (*)
17/12/2021 29/12/2021 BRADESPAR,S.A. BRADESPAR PREFERENTES BRBRAPACNPR2 A cuenta 2021 Ord. (*)
14/12/2021 16/12/2021 ATRESMEDIA CORP. DE MEDIOS DE COM. S.A. ATRESMEDIA CORP. DE MEDIOS DE COM. ES0109427734 A cuenta 2021 Ord. (*)
14/12/2021 16/12/2021 PRIM, S.A. PRIM ES0170884417 A cuenta 2021 Ord. (*)
07/12/2021 10/12/2021 HISPANOTELS INVERSIONES SOCIMI, S.A. HISPANOTELS INVERSIONES ES0105379004 RESERVAS 2021 Ord. (*)
30/11/2021 02/12/2021 ALTIA CONSULTORES, S.A. ALTIA CONSULTORES ES0113312005 A cuenta 2021 Ord. (*)
30/11/2021 02/12/2021 WEALTH RENDITE, SICAV, S.

In [36]:
columns=[]

for element in table.find_elements(By.TAG_NAME, "th"):    
    columns.append(element.text)
    
print(columns)

['Fecha\nDescuento', 'Fecha\nAbono', 'Emisora', 'Valor', 'ISIN Valor', 'Ejercicio', 'Tipo', 'Importe Bruto']


In [37]:
results=[]

for element in table.find_elements(By.TAG_NAME, "td"):    
    results.append(element.text)

print(results)

['21/12/2021', '23/12/2021', 'CLINICA BAVIERA, S.A.', 'CLINICA BAVIERA', 'ES0119037010', 'A cuenta 2021', 'Ord.', '(*)', '20/12/2021', '22/12/2021', 'VISCOFAN, S.A.', 'VISCOFAN', 'ES0184262212', 'A cuenta 2021', 'Ord.', '(*)', '17/12/2021', '21/12/2021', 'ENAGAS, S.A.', 'ENAGAS', 'ES0130960018', 'A cuenta 2021', 'Ord.', '(*)', '17/12/2021', '29/12/2021', 'BRADESPAR,S.A.', 'BRADESPAR ORDINARIAS', 'BRBRAPACNOR5', 'A cuenta 2021', 'Ord.', '(*)', '17/12/2021', '29/12/2021', 'BRADESPAR,S.A.', 'BRADESPAR PREFERENTES', 'BRBRAPACNPR2', 'A cuenta 2021', 'Ord.', '(*)', '14/12/2021', '16/12/2021', 'ATRESMEDIA CORP. DE MEDIOS DE COM. S.A.', 'ATRESMEDIA CORP. DE MEDIOS DE COM.', 'ES0109427734', 'A cuenta 2021', 'Ord.', '(*)', '14/12/2021', '16/12/2021', 'PRIM, S.A.', 'PRIM', 'ES0170884417', 'A cuenta 2021', 'Ord.', '(*)', '07/12/2021', '10/12/2021', 'HISPANOTELS INVERSIONES SOCIMI, S.A.', 'HISPANOTELS INVERSIONES', 'ES0105379004', 'RESERVAS 2021', 'Ord.', '(*)', '30/11/2021', '02/12/2021', 'ALTIA C

In [38]:
data = {}
for i in range(8):
    data[columns[i]]=results[i::8]

df = pd.DataFrame(data)

In [39]:
df

,Fecha\nDescuento,Fecha\nAbono,Emisora,Valor,ISIN Valor,Ejercicio,Tipo,Importe Bruto
0,21/12/2021,23/12/2021,"CLINICA BAVIERA, S.A.",CLINICA BAVIERA,ES0119037010,A cuenta 2021,Ord.,(*)
1,20/12/2021,22/12/2021,"VISCOFAN, S.A.",VISCOFAN,ES0184262212,A cuenta 2021,Ord.,(*)
2,17/12/2021,21/12/2021,"ENAGAS, S.A.",ENAGAS,ES0130960018,A cuenta 2021,Ord.,(*)
3,17/12/2021,29/12/2021,"BRADESPAR,S.A.",BRADESPAR ORDINARIAS,BRBRAPACNOR5,A cuenta 2021,Ord.,(*)
4,17/12/2021,29/12/2021,"BRADESPAR,S.A.",BRADESPAR PREFERENTES,BRBRAPACNPR2,A cuenta 2021,Ord.,(*)
5,14/12/2021,16/12/2021,ATRESMEDIA CORP. DE MEDIOS DE COM. S.A.,ATRESMEDIA CORP. DE MEDIOS DE COM.,ES0109427734,A cuenta 2021,Ord.,(*)
6,14/12/2021,16/12/2021,"PRIM, S.A.",PRIM,ES0170884417,A cuenta 2021,Ord.,(*)
7,07/12/2021,10/12/2021,"HISPANOTELS INVERSIONES SOCIMI, S.A.",HISPANOTELS INVERSIONES,ES0105379004,RESERVAS 2021,Ord.,(*)
8,30/11/2021,02/12/2021,"ALTIA CONSULTORES, S.A.",ALTIA CONSULTORES,ES0113312005,A cuenta 2021,Ord.,(*)
9,30/11/2021,02/12/2021,"WEALTH RENDITE, SICAV, S.A.",WEALTH RENDITE,ES0145837037,RESERVAS 2021,Ord.,(*)


In [40]:
df.to_csv('Dividents.csv')
browser.close()
quit()

## Level 2

### Exercise 2

#### Information about files from Kaggle in word document

Decsription of dataset:
https://github.com/dominikapetru/web_scraping/blob/main/Description.docx

### Dataset Indices Summary from Madrid Stock Exchange

#### Context

The dataset consists of set of price data for indices tracking stock exchanges from Spain. The data was all collected from Madrid Stock Exchange website 17.11.2021 at 9:50AM.

Prices are quoted in euros.

This dataset was created is a result of webscraping test and is designed to be a playground for pre-processing and technical analysis.

#### Content

Indices.csv: prices for indices tracking stock exchanges

The categories of this dataset: 
- Nombre - Name of inex
- Anterior - Previous price
- Último - Previous price
- % Dif. - Difference of prices in %
- Máximo - High
- Mínimo - Low
- Fecha - Date
- Hora - Hour
- % Dif.Año 2021 - Difference of prices for 2021 in %

#### Acknowledgements

- Source: https://www.bolsamadrid.es
- Github: https://github.com/dominikapetru/web_scraping

In [3]:
df = pd.read_csv('Indices.csv')
df.head(5)

,Unnamed: 0,Nombre,Anterior,Último,% Dif.,Máximo,Mínimo,Fecha,Hora,% Dif.Año 2021
0,0,IBEX 35®,"8.402,70","8.455,20","0,62","8.562,00","8.431,70",29/11/2021,17:38:00,"4,73"
1,1,IBEX 35® con Dividendos,"25.846,10","26.007,70","0,63","26.336,20","25.935,40",29/11/2021,17:38:00,"7,20"
2,2,IBEX MEDIUM CAP®,"12.992,60","13.071,80","0,61","13.183,00","13.029,90",29/11/2021,17:38:00,"2,80"
3,3,IBEX SMALL CAP®,"7.862,50","7.874,30","0,15","7.966,40","7.866,60",29/11/2021,17:38:00,"-2,76"
4,4,IBEX 35® Bancos,"435,20","434,30","-0,21","445,90","433,40",29/11/2021,17:38:00,"15,47"


In [4]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Nombre          78 non-null     object
 1   Anterior        78 non-null     object
 2   Último          78 non-null     object
 3   % Dif.          78 non-null     object
 4   Máximo          78 non-null     object
 5   Mínimo          78 non-null     object
 6   Fecha           78 non-null     object
 7   Hora            78 non-null     object
 8   % Dif.Año 2021  78 non-null     object
dtypes: object(9)
memory usage: 5.6+ KB


In [6]:
df['Anterior'] = df['Anterior'].str.replace('.','').str.replace(',','.').astype(float)
df['Último'] = df['Último'].str.replace('.','').str.replace(',','.').astype(float)
df['% Dif.'] = df['% Dif.'].str.replace(',','.')
df['Máximo'] = df['Máximo'].str.replace('.','').str.replace(',','.').astype(float)
df['Mínimo'] = df['Mínimo'].str.replace('.','').str.replace(',','.').astype(float)
df['% Dif.Año 2021'] = df['% Dif.Año 2021'].str.replace(',','.')

df.head(5)

,Nombre,Anterior,Último,% Dif.,Máximo,Mínimo,Fecha,Hora,% Dif.Año 2021
0,IBEX 35®,8402.7,8455.2,0.62,8562.0,8431.7,29/11/2021,17:38:00,4.73
1,IBEX 35® con Dividendos,25846.1,26007.7,0.63,26336.2,25935.4,29/11/2021,17:38:00,7.20
2,IBEX MEDIUM CAP®,12992.6,13071.8,0.61,13183.0,13029.9,29/11/2021,17:38:00,2.80
3,IBEX SMALL CAP®,7862.5,7874.3,0.15,7966.4,7866.6,29/11/2021,17:38:00,-2.76
4,IBEX 35® Bancos,435.2,434.3,-0.21,445.9,433.4,29/11/2021,17:38:00,15.47


In [7]:
list = len(df)
list

78

In [8]:
#negative values - conversion to float
for i in range (list):
    df['% Dif.'] = df.at[i,'% Dif.'].replace('\U00002013', '-')
    df['% Dif.Año 2021'] = df.at[i,'% Dif.Año 2021'].replace('\U00002013', '-')

In [9]:
df['% Dif.'].astype(float)
df['% Dif.Año 2021'].astype(float)

df.head(5)

,Nombre,Anterior,Último,% Dif.,Máximo,Mínimo,Fecha,Hora,% Dif.Año 2021
0,IBEX 35®,8402.7,8455.2,0.62,8562.0,8431.7,29/11/2021,17:38:00,4.73
1,IBEX 35® con Dividendos,25846.1,26007.7,0.62,26336.2,25935.4,29/11/2021,17:38:00,4.73
2,IBEX MEDIUM CAP®,12992.6,13071.8,0.62,13183.0,13029.9,29/11/2021,17:38:00,4.73
3,IBEX SMALL CAP®,7862.5,7874.3,0.62,7966.4,7866.6,29/11/2021,17:38:00,4.73
4,IBEX 35® Bancos,435.2,434.3,0.62,445.9,433.4,29/11/2021,17:38:00,4.73


Clean dataset ready for data analysis.

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Nombre          78 non-null     object 
 1   Anterior        78 non-null     float64
 2   Último          78 non-null     float64
 3   % Dif.          78 non-null     object 
 4   Máximo          78 non-null     float64
 5   Mínimo          78 non-null     float64
 6   Fecha           78 non-null     object 
 7   Hora            78 non-null     object 
 8   % Dif.Año 2021  78 non-null     object 
dtypes: float64(4), object(5)
memory usage: 5.6+ KB
